In [1]:
from pydantic import BaseModel
from typing import Tuple

class Person(BaseModel):
  firstname: str
  lastname: str
  height: Tuple[int, int]



In [3]:
me = Person(firstname="Alpri", lastname="Else", height=(5, 11))

In [4]:
me

Person(firstname='Alpri', lastname='Else', height=(5, 11))

In [6]:
me.json()

/var/folders/kt/7gdyyk6d75lbw1xwdqkrbrjw0000gn/T/ipykernel_47559/664548708.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  me.json()


'{"firstname":"Alpri","lastname":"Else","height":[5,11]}'

In [7]:
me.model_dump()

{'firstname': 'Alpri', 'lastname': 'Else', 'height': (5, 11)}

In [8]:
me.model_dump()['firstname']

'Alpri'

In [9]:
Person.model_json_schema()

{'properties': {'firstname': {'title': 'Firstname', 'type': 'string'},
  'lastname': {'title': 'Lastname', 'type': 'string'},
  'height': {'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'integer'}, {'type': 'integer'}],
   'title': 'Height',
   'type': 'array'}},
 'required': ['firstname', 'lastname', 'height'],
 'title': 'Person',
 'type': 'object'}

In [21]:
from typing_extensions import TypedDict

class Pastery(TypedDict):
  name: str
  bake_duration_mins: int

croissant: Pastery = { 'name': 'Croissant', 'bake_duration_mins': 10}

In [22]:
croissant2: Pastery = { 'name': 'Croissant', 'bake_duration_mins': '10'}

In [23]:
croissant['name']

'Croissant'

In [24]:
croissant2['bake_duration_mins']

'10'

In [25]:
from pydantic import TypeAdapter

pastery_adapter = TypeAdapter(Pastery)




In [29]:
pastery_adapter.validate_python({'name': 'Croissant', 'bake_duration_mins': '10'})

{'name': 'Croissant', 'bake_duration_mins': 10}

In [31]:
pastery_adapter.validate_python({'name': 'Croissant', 'bake_duration_mins': '10'}, strict=True)

ValidationError: 1 validation error for typed-dict
bake_duration_mins
  Input should be a valid integer [type=int_type, input_value='10', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_type

In [28]:
pastery_adapter.validate_python({'name': 'Croissant', 'bake_duration_mins': 10})

{'name': 'Croissant', 'bake_duration_mins': 10}

In [35]:
from datetime import datetime
class Meeting(BaseModel):
  time: datetime



In [37]:
Meeting.model_validate({'time': 'hi'})

ValidationError: 1 validation error for Meeting
time
  Input should be a valid datetime, input is too short [type=datetime_parsing, input_value='hi', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/datetime_parsing

In [38]:
from pydantic import field_validator

class MeetingV2(BaseModel):
  time: datetime

  @field_validator('time', mode='wrap')
  def when_now(cls, input_value, handler):
    if input_value == 'now':
      return datetime.now()
    
    when = handler(input_value)
    return when


In [40]:
MeetingV2(time='now')

MeetingV2(time=datetime.datetime(2023, 11, 25, 12, 51, 54, 503145))

In [11]:
from pydantic import BaseModel, computed_field

class TestModel(BaseModel):
  firstname: str
  lastname: str
  _secret: str

  @computed_field(repr=False)
  def fullname(self) -> str:
    return f'{self.firstname} {self.lastname}'
  
  def fullname_v2(self) -> str:
    return f'{self.firstname} {self.lastname}'


x = TestModel(firstname='Alpri', lastname='Else')
x.dict()


/var/folders/kt/7gdyyk6d75lbw1xwdqkrbrjw0000gn/T/ipykernel_49160/2003782499.py:17: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  x.dict()


{'firstname': 'Alpri', 'lastname': 'Else', 'fullname': 'Alpri Else'}